## Pfizer Vaccine EDA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import boxcox
from collections import Counter
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/pfizer-vaccine-tweets/vaccination_tweets.csv')
df = df.drop('id', axis=1)

## Data cleaning

In [ ]:
df['user_description'] = df['user_description'].fillna('NaN')
df['user_location'] = df['user_location'].fillna('NaN')
df['hashtags'] = df['hashtags'].fillna('NaN')
df['source'] = df['source'].fillna('NaN')

locations = pd.Series(Counter(df['user_location'])).sort_values(ascending=False)
locations = locations.drop('NaN')[locations>5]

source = Counter(df['source'])
source = pd.Series(source).sort_values(ascending=False)

verified = Counter(df['user_verified'])
verified = pd.Series(verified).sort_values(ascending=False)

name = Counter(df['user_name'])
name = pd.Series(name).sort_values(ascending=False)
name = name[name>4]

hashtags = []
for i in df['hashtags']:
    for j in i.translate(str.maketrans('', '', string.punctuation)).split(' '):
        hashtags.append(j)
hashtags = Counter(hashtags)
hashtags = pd.Series(hashtags).sort_values(ascending=False)
hashtags = hashtags[hashtags>8]

followers = df.sort_values(by='user_followers', ascending=False)
followers_i = followers['user_name'].drop_duplicates().keys()[:15]

friends = df.sort_values(by='user_friends', ascending=False)[:15]
friends_i = friends['user_name'].drop_duplicates().keys()

favourites = df.sort_values(by='user_favourites', ascending=False)
favourites_i = favourites['user_name'].drop_duplicates().keys()[:15]

## Visualisation procedures

### Bar chart

In [ ]:
def barplot(x, y, title, xlabel, ylabel, capsize=12, fontsize=12, labelsize=9, 
            palette='twilight', rotation=0, figsize=(13, 12), bartext_size=15):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    bars = sns.barplot(x, y, palette=palette, capsize=capsize)
    for bar in bars.patches:
        bars.annotate(format(bar.get_height(), '.0f'), 
                  (bar.get_x()+bar.get_width()/2., bar.get_height()), ha='center', va='bottom',
                    size=bartext_size)
    
    plt.title(title, fontsize=fontsize)
    plt.xlabel(xlabel, fontsize=fontsize)
    plt.ylabel(ylabel, fontsize=fontsize)
    plt.xticks(rotation=rotation)
    plt.tick_params(axis='x', labelsize=labelsize)
    plt.show()

### Histogram

In [ ]:
def dist(column, name, colour):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    num_words = pd.Series([len(i.split(' ')) for i in df[column]])
    
    pd.Series(num_words).hist(ax=ax1, color=colour)
    pd.DataFrame((num_words+1).transform(np.log)).hist(ax=ax2, color=colour)
    pd.DataFrame(boxcox(num_words+1)[0]).hist(ax=ax3, color=colour)
    
    ax1.set_title('Normal')
    ax2.set_title('Log transform')
    ax3.set_title('Box cox')
    
    for ax in (ax1, ax2, ax3):
        ax.set_xlabel(name + ' by length')
        ax.set_ylabel('Number of ' + name)
    
    plt.show()

### Pie plots

In [ ]:
def piechart(x, y, title, figsize=(10, 13)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    plt.pie(x, labels=y, autopct=lambda p:f'{p:.2f}%')
    plt.title(title)
    plt.show()

### Line graph

In [ ]:
def time_plot(time, title, x, y):
    count = Counter(time)
    count = pd.Series(dict(zip(count.values(), [int(i) for i in count.keys()]))).sort_values()

    fig, ax = plt.subplots(1, 1, figsize=(13, 10))
    ax.plot([str(i) for i in count], count.keys())
    ax.set_title(title)
    ax.set_xlabel(x)
    ax.set_ylabel(y)
    plt.show()

## Bar charts

### Location

In [ ]:
barplot(locations.keys(), locations, title='Tweets per location', xlabel='Location', 
        ylabel='Number of tweets', rotation=90, bartext_size=10)

### Source

In [ ]:
barplot(source.keys(), source, title='Tweets per source', xlabel='Source', 
        ylabel='Number of tweets', palette='Accent', rotation=90)

### Username

In [ ]:
barplot(name.keys(), name, 'People who tweeted', 'Username', 'Number of tweets', rotation=90,
       palette='autumn', bartext_size=11)

### Hashtags

In [ ]:
barplot(hashtags.keys(), hashtags, 'Hashtags for tweets', 'Hashtags', 'Number of tweets',
       rotation=90, bartext_size=10)

### Verified

In [ ]:
barplot(verified.keys(), verified, 'Verified twitter acounts', 
        'Whether an account is verified or not', 'Number of accounts')

## Distribution charts

### User description

In [ ]:
dist('user_description', 'user descriptions', 'blue')
df['description_length'] = (pd.Series([len(i.split(' ')) 
                                       for i in df['user_description']])+1).transform(np.log)

### Tweets

In [ ]:
dist('text', 'tweets', 'pink')
df['text_length'] = (boxcox(pd.Series([len(i.split(' ')) for i in df['text']]))[0]+1)

## Pie plots

### Followers

In [ ]:
piechart(followers['user_followers'][followers_i], followers['user_name'][followers_i],
        'Followers per source')

### Friends

In [ ]:
piechart(friends['user_friends'][friends_i], friends['user_name'][friends_i],
         'Number of friends per source')

### Favourites

In [ ]:
piechart(favourites['user_favourites'][favourites_i], favourites['user_name'][favourites_i],
         'Amount of favourites per source')

## Line graphs

### Days

In [ ]:
time_plot([str(i)[8:10] for i in df['date']], 'Number of tweets over the days', 
          'Day in December', 'Number of tweets')

### Hours

In [ ]:
time_plot([str(i)[11:13] for i in df['date']], 'Number of tweets over the hours', 
          'Hour when tweet was released', 'Number of tweets')

## Classifying the data

In [ ]:
X = df[['user_followers', 'user_friends', 'user_favourites', 'retweets', 
        'favorites', 'description_length', 'text_length']]
df = df.drop('is_retweet', axis=1)

In [ ]:
sns.heatmap(df.corr(), annot=True)
plt.title('Correlation')
plt.show()

In [ ]:
y = list()
for i in df['source']:
    if i == 'Twitter for iPhone':
        y.append(i)
    elif i == 'Twitter for Android':
        y.append(i)
    elif i == 'Twitter Web App':
        y.append(i)
    else:
        y.append('Other')

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(X)

eva = pca.explained_variance_
plt.bar(range(len(eva)), eva)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
for classifier in [[RandomForestClassifier(), 'Random Forest'], [XGBClassifier(), 'XGBoost']]:
    model = classifier[0]
    model.fit(X_train, y_train)
    print(classifier[1])
    print(model.score(X_test, y_test))
    print(cross_val_score(model, X_test, y_test).mean())

### Thank you for reading my notebook.

### If you enjoyed this notebook and found it helpful, please upvote it and give feedback as it would help me make more of these.